In [59]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import polars as pl
import pytomlpp as toml
import pandas as pd
from data_preprocessing import getXY, data_normalizeation
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from tqdm import tqdm

In [60]:
cfg = toml.load('/home/gyli/GP_HP/cfg/feature_selection.toml')
data_path = cfg['data_path']
variance_filter_thres = cfg['variance_filter_thres']
raw_x, raw_y = getXY(data_path)
# raw_x = data_normalizeation(raw_x)
future_names = raw_x.columns
X, y = raw_x.select(['年龄', '淋巴细胞数', '白蛋白', '活化部分凝血活酶时间', '低密度脂蛋白', '钠', '中性粒细胞数', '甘油三酯', '凝血酶原时间', '红细胞平均体积', '总胆红素']).to_pandas(), np.ravel(raw_y.to_pandas())

In [64]:
pl.read_csv(data_path, dtypes={'年龄' : pl.Float64}).with_columns(pl.col('年龄').cast(pl.Int64)).describe()

statistic,年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原,是否高血压
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0,33287.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",51.40271,4.652154,2.029147,0.160119,0.013169,69.151504,42.608433,26.54341,11.174038,3.127396,4.056288,141.334061,2.309005,5.273337,1.138146,1.977214,1.301968,2.947296,92.164613,12.734557,0.999325,32.961164,3.652884,0.487127
"""std""",16.02117,2.184545,0.684662,0.139704,0.019416,7.420115,6.295756,3.979835,6.517321,2.457711,0.387083,2.367334,0.145477,3.092034,0.633057,1.569655,0.3263,0.859456,6.294701,1.371936,0.132464,3.934606,0.642092,0.499842
"""min""",0.0,0.05,0.11,0.0,-0.007842,27.3,11.6,11.2,0.5,0.081103,2.1,96.0,1.24,-0.917334,0.1,-1.517163,0.14,0.01,48.6,0.4677637,-0.189686,0.0,0.694933,0.0
"""25%""",39.0,3.447011,1.68,0.08,0.007313,64.9,39.1,24.0,7.5,2.2,3.9,140.268747,2.24,3.9,0.8,1.16,1.1,2.45,90.589882,12.4,0.966638,31.978493,3.41,0.0
"""50%""",51.0,4.125127,2.088231,0.159508,0.01,70.3,43.856382,26.37876,10.3,2.885863,4.060188,141.630883,2.33,4.9,1.0,1.72687,1.29,2.926007,92.1,12.728788,0.995275,32.967713,3.59973,0.0
"""75%""",63.0,5.20055,2.23,0.18032,0.015283,74.3,47.2,28.7,13.2,3.6,4.2,142.424737,2.4,5.8,1.3,2.210309,1.446983,3.35,94.2,13.062113,1.024223,33.934014,3.79116,1.0
"""max""",99.0,51.36,52.5,6.35,1.6,124.1,59.6,78.1,291.4,161.7,19.5,164.6,4.37,64.0,19.0,65.44,4.73,15.53,133.6,66.1,8.18,119.0,11.04,1.0


In [69]:
pl.read_csv(data_path, dtypes={'年龄' : pl.Float64}).with_columns(pl.col('年龄').cast(pl.Int64))['是否高血压'].value_counts()

是否高血压,count
i64,u32
1,16215
0,17072


In [61]:
raw_x.describe()

statistic,年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",52.254056,4.567131,2.090111,0.170863,0.017453,70.440742,43.618026,26.823058,11.517256,3.229566,4.082569,141.472249,2.334196,5.272611,1.085323,1.977849,1.291036,2.957736,91.495374,12.660115,0.998116,32.570699,3.596984
"""std""",15.386269,1.994131,0.664631,0.141238,0.019557,6.839072,5.926141,3.912165,5.942347,2.291735,0.37868,2.276023,0.131797,2.742084,0.555307,1.602298,0.321843,0.878035,5.036598,1.284763,0.117857,3.618886,0.549035
"""min""",1.0,0.05,0.13,0.01,0.000287,27.3,11.6,11.6,0.8,0.1,2.1,106.4,1.29,0.2,0.1,0.07,0.149165,0.01,57.6,5.198928,0.297887,13.710337,0.81
"""25%""",41.0,3.519042,1.85,0.11,0.01,67.0,40.7,24.3,8.0,2.3,3.914825,140.6,2.274896,4.0,0.8,1.13,1.09,2.42,90.796167,12.4,0.969918,31.946807,3.408363
"""50%""",52.0,4.11,2.117365,0.164643,0.012574,71.4,44.8,26.6,10.6,2.993746,4.083333,141.771598,2.35,5.0,1.0,1.695664,1.28,2.929331,91.958932,12.71707,0.996111,32.902395,3.587174
"""75%""",64.0,4.98,2.23,0.181875,0.02,74.9,47.7,29.0,13.5,3.7,4.2,142.5,2.41,5.8,1.3,2.2,1.431957,3.39,93.117697,13.022838,1.023449,33.735237,3.7556
"""max""",99.0,51.36,52.5,6.35,1.6,102.6,59.6,66.8,236.3,144.3,19.5,164.6,4.37,63.3,19.0,35.59,4.73,15.13,133.6,37.1,3.79,97.9,11.04


In [62]:
pl.from_pandas(X).describe()

statistic,年龄,淋巴细胞数,白蛋白,活化部分凝血活酶时间,低密度脂蛋白,钠,中性粒细胞数,甘油三酯,凝血酶原时间,红细胞平均体积,总胆红素
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0,24841.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",52.254056,2.090111,43.618026,32.570699,2.957736,141.472249,4.567131,1.977849,12.660115,91.495374,11.517256
"""std""",15.386269,0.664631,5.926141,3.618886,0.878035,2.276023,1.994131,1.602298,1.284763,5.036598,5.942347
"""min""",1.0,0.13,11.6,13.710337,0.01,106.4,0.05,0.07,5.198928,57.6,0.8
"""25%""",41.0,1.85,40.7,31.946807,2.42,140.6,3.519042,1.13,12.4,90.796167,8.0
"""50%""",52.0,2.117365,44.8,32.902395,2.929331,141.771598,4.11,1.695664,12.71707,91.958932,10.6
"""75%""",64.0,2.23,47.7,33.735237,3.39,142.5,4.98,2.2,13.022838,93.117697,13.5
"""max""",99.0,52.5,59.6,97.9,15.13,164.6,51.36,35.59,37.1,133.6,236.3


In [48]:
knn_model = KNeighborsClassifier(n_neighbors=3)

# 创建随机森林模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 创建GBDT模型
gbdt_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# 创建Pipeline来对数据进行归一化处理
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 归一化处理
    ('model', knn_model)  # 在这里更改模型
])

# 对K近邻模型进行交叉验证
knn_scores = cross_val_score(pipeline, X, y, cv=5)
print("K近邻模型交叉验证结果:", knn_scores,'均值',knn_scores.mean(),'方差',knn_scores.std())

# 创建随机森林模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 创建GBDT模型
gbdt_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# 对随机森林模型进行交叉验证
rf_scores = cross_val_score(rf_model, X, y, cv=5)
print("随机森林模型交叉验证结果:", rf_scores,'均值',rf_scores.mean(),'方差',rf_scores.std())

# 对GBDT模型进行交叉验证
gbdt_scores = cross_val_score(gbdt_model, X, y, cv=5)
print("GBDT模型交叉验证结果:", gbdt_scores,'均值',gbdt_scores.mean(),'方差',gbdt_scores.mean())



K近邻模型交叉验证结果: [0.75005031 0.60487118 0.80595813 0.77355072 0.63486312] 均值 0.7138586936267337 方差 0.07933959601975767
随机森林模型交叉验证结果: [0.88206883 0.70229469 0.91626409 0.90257649 0.7071256 ] 均值 0.8220659392581833 方差 0.09644897164433966
GBDT模型交叉验证结果: [0.74763534 0.54066023 0.87902576 0.85386473 0.61976651] 均值 0.7281905138752375 方差 0.7281905138752375


In [31]:
# knn_df = pd.DataFrame(columns=['n_neighbors', 'weights', 'p_val', 'cross_val_score', 'mean_score', 'std'])
knn_dfs = []
n_neighbors = [i for i in range(1, 20)]
weights = ['uniform', 'distance']
p_vals = [i for i in range(1,10)]
for k in n_neighbors:
    for w in weights:
        for p in p_vals:
            knn_model = KNeighborsClassifier(n_neighbors=k, weights=w,p=p)
            pipeline = Pipeline([
                ('scaler', StandardScaler()),  # 归一化处理
                ('model', knn_model)  # 在这里更改模型
            ])
            # 对K近邻模型进行交叉验证
            knn_scores = cross_val_score(pipeline, X, y, cv=5)
            row = {'n_neighbors' : k, 'weights' : w, 'p_val' : p, 'mean_score' : knn_scores.mean(), 'std' : knn_scores.std()}
            for i, acc in enumerate(knn_scores):
                row[f'cross_val_score_{i+1}'] = knn_scores[i]
            # knn_df = knn_df.add(row, ignore_index=True)
            knn_dfs.append(pd.DataFrame(row, index=[0]))
knn_df = pd.concat(knn_dfs)
knn_df.to_csv('/home/gyli/GP_HP/result/knn_results.csv')

In [32]:
knn_df

,n_neighbors,weights,p_val,mean_score,std,cross_val_score_1,cross_val_score_2,cross_val_score_3,cross_val_score_4,cross_val_score_5
0,1,uniform,1,0.790223,0.069729,0.849265,0.722826,0.857689,0.831320,0.690016
0,1,uniform,2,0.785674,0.071945,0.846448,0.713969,0.854267,0.829911,0.683776
0,1,uniform,3,0.785997,0.070294,0.843429,0.713969,0.856280,0.827899,0.688406
0,1,uniform,4,0.785071,0.069902,0.844033,0.714372,0.854469,0.825081,0.687399
0,1,uniform,5,0.785232,0.070526,0.844637,0.714372,0.855676,0.825081,0.686393
...,...,...,...,...,...,...,...,...,...,...
0,19,distance,5,0.790385,0.096057,0.829744,0.682367,0.908615,0.859702,0.671498
0,19,distance,6,0.788413,0.095221,0.828939,0.682166,0.904589,0.857085,0.669283
0,19,distance,7,0.787688,0.095603,0.826524,0.680757,0.904589,0.857890,0.668680
0,19,distance,8,0.787648,0.094858,0.826927,0.679952,0.904187,0.856079,0.671095


In [58]:
rf_dfs = []
# n_estimators = [i for i in range(100, 300, 100)]
# max_depth = [i for i in range(2, 3)]
# min_samples_leaf = [2]
# min_samples_split = [2]
# max_features = [None, 'sqrt', 'log2']
# min_samples_leaf = [i for i in range(2, 3)]
# min_samples_split = [i for i in range(2, 3)]
n_estimators = [i for i in range(200, 300, 10)]
max_depth = [i for i in range(20, 12)]
min_samples_leaf = [1]
min_samples_split = [2]
max_features = [None, 'sqrt', 'log2']

for n in n_estimators:
    for md in max_depth:
        for ml in  min_samples_leaf:
            for ms in min_samples_split:
                for mf in max_features:
                    rf_model = RandomForestClassifier(n_estimators=n, max_depth=md, min_samples_leaf=ml, min_samples_split=ms, max_features=mf, random_state=42)
                    rf_scores = cross_val_score(rf_model, X, y, cv=5)
                    row = {'n_estimators' : n, 'max_depth' : md, 'min_samples_leaf' : ml, 'min_samples_split' : ms, 'max_features' : mf, 'mean_score' : rf_scores.mean(), 'std' : rf_scores.std()}
                    for i, acc in enumerate(rf_scores):
                        row[f'cross_val_score_{i+1}'] = rf_scores[i]
                    rf_dfs.append(pd.DataFrame(row, index=[0]))
rf_df = pd.concat(rf_dfs)
rf_df.to_csv('/home/gyli/GP_HP/result/rf_results.csv')

ValueError: No objects to concatenate

In [50]:
[i for i in range(8, 11)]

[8, 9, 10]

In [39]:
gbdt_dfs = []
# n_estimators = [i for i in range(100, 300, 100)]
# learning_rate = [i * 0.05 for i in range(0, 20, 10)]
# subsample = [i*0.05+0.5 for i in range(0, 6, 3)]
n_estimators = [i for i in range(10, 200, 10)]
learning_rate = [round(i * 0.1, 3) for i in range(1, 5, 1)]
# subsample = [round(i*0.1+0.4, 3) for i in range(1, 3, 1)]
# loss = ['log_loss','exponential']
loss = ['log_loss']
for n in n_estimators:
    for lr in learning_rate:
        for l in loss:
            gbdt_model = GradientBoostingClassifier(n_estimators=n, learning_rate=lr, loss=l, random_state=42)
            gbdt_scores = cross_val_score(gbdt_model, X, y, cv=5)
            row = {'n_estimators' : n, 'learning_rate' : lr, 'loss' : l, 'mean_score' : gbdt_scores.mean(), 'std' : gbdt_scores.std()}
            for i, acc in enumerate(gbdt_scores):
                row[f'cross_val_score_{i+1}'] = gbdt_scores[i]
            gbdt_dfs.append(pd.DataFrame(row, index=[0]))
gbdt_df = pd.concat(gbdt_dfs)
gbdt_df.to_csv('/home/gyli/GP_HP/result/gbdt_results.csv')

In [40]:
gbdt_df

,n_estimators,learning_rate,loss,mean_score,std,cross_val_score_1,cross_val_score_2,cross_val_score_3,cross_val_score_4,cross_val_score_5
0,10,0.1,log_loss,0.721630,0.130036,0.714027,0.555354,0.907206,0.821457,0.610105
0,10,0.2,log_loss,0.720784,0.130458,0.722278,0.541868,0.888889,0.836755,0.614130
0,10,0.3,log_loss,0.722918,0.133600,0.725699,0.539251,0.889694,0.847424,0.612520
0,10,0.4,log_loss,0.723642,0.130728,0.733347,0.545491,0.887681,0.841385,0.610306
0,20,0.1,log_loss,0.722878,0.133203,0.721876,0.544283,0.899960,0.836957,0.611312
...,...,...,...,...,...,...,...,...,...,...
0,180,0.4,log_loss,0.739904,0.114846,0.776011,0.575282,0.859903,0.851651,0.636675
0,190,0.1,log_loss,0.730002,0.126674,0.755685,0.548108,0.871779,0.851852,0.622585
0,190,0.2,log_loss,0.736604,0.122076,0.768565,0.560386,0.870974,0.851852,0.631240
0,190,0.3,log_loss,0.736885,0.116884,0.764339,0.568639,0.868559,0.846014,0.636876


In [49]:
pl.from_pandas(rf_df).sort(by='mean_score', descending=True)

n_estimators,max_depth,min_samples_leaf,min_samples_split,max_features,mean_score,std,cross_val_score_1,cross_val_score_2,cross_val_score_3,cross_val_score_4,cross_val_score_5
i64,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64
190,10,2,2,"""sqrt""",0.754115,0.127033,0.790501,0.576087,0.897947,0.869163,0.636876
190,10,2,2,"""log2""",0.754115,0.127033,0.790501,0.576087,0.897947,0.869163,0.636876
180,10,2,2,"""sqrt""",0.753712,0.126985,0.790099,0.576288,0.897746,0.868559,0.63587
180,10,2,2,"""log2""",0.753712,0.126985,0.790099,0.576288,0.897746,0.868559,0.63587
160,10,2,2,"""sqrt""",0.753591,0.126738,0.790501,0.575886,0.898148,0.866747,0.636675
…,…,…,…,…,…,…,…,…,…,…,…
60,2,2,2,null,0.697959,0.142121,0.695915,0.486514,0.890298,0.806159,0.61091
80,2,2,2,null,0.697919,0.142281,0.694506,0.486916,0.891103,0.806361,0.610709
90,2,2,2,null,0.697838,0.142369,0.694707,0.486514,0.891103,0.806159,0.610709


In [ ]:
n_estimators = [i for i in range(5, 300, 5)]
learning_rate = [round(i * 0.05, 3) for i in range(1, 20, 1)]
subsample = [round(i*0.05+0.4, 3) for i in range(0, 8, 1)]

In [ ]:
subsample

[0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]

In [72]:
pl.read_csv('/home/gyli/GP_HP/result/gbdt_results.csv').sort(by='mean_score', descending=True)

,n_estimators,learning_rate,loss,mean_score,std,cross_val_score_1,cross_val_score_2,cross_val_score_3,cross_val_score_4,cross_val_score_5
i64,i64,f64,str,f64,f64,f64,f64,f64,f64,f64
0,190,0.4,"""log_loss""",0.740549,0.114374,0.7742,0.577295,0.859903,0.853663,0.637681
0,180,0.4,"""log_loss""",0.739904,0.114846,0.776011,0.575282,0.859903,0.851651,0.636675
0,170,0.4,"""log_loss""",0.739139,0.115645,0.775206,0.573068,0.859501,0.852254,0.635668
0,160,0.4,"""log_loss""",0.737489,0.117273,0.774804,0.569042,0.861715,0.849436,0.632448
0,150,0.4,"""log_loss""",0.737328,0.118521,0.772791,0.567633,0.86252,0.852456,0.63124
…,…,…,…,…,…,…,…,…,…,…
0,10,0.4,"""log_loss""",0.723642,0.130728,0.733347,0.545491,0.887681,0.841385,0.610306
0,10,0.3,"""log_loss""",0.722918,0.1336,0.725699,0.539251,0.889694,0.847424,0.61252
0,20,0.1,"""log_loss""",0.722878,0.133203,0.721876,0.544283,0.89996,0.836957,0.611312


In [1]:
knn_predictions = pipeline.predict(X_test)
# svm_predictions = svm_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
gbdt_predictions = gbdt_model.predict(X_test)

# 计算准确率
knn_accuracy = accuracy_score(y_test, knn_predictions)
# svm_accuracy = accuracy_score(y_test, svm_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)
gbdt_accuracy = accuracy_score(y_test, gbdt_predictions)

# 计算精确率
knn_precision = precision_score(y_test, knn_predictions)
# svm_precision = precision_score(y_test, svm_predictions)
rf_precision = precision_score(y_test, rf_predictions)
gbdt_precision = precision_score(y_test, gbdt_predictions)

# 计算召回率
knn_recall = recall_score(y_test, knn_predictions)
# svm_recall = recall_score(y_test, svm_predictions)
rf_recall = recall_score(y_test, rf_predictions)
gbdt_recall = recall_score(y_test, gbdt_predictions)

# 计算 F1 分数
knn_f1_score = f1_score(y_test, knn_predictions)
# svm_f1_score = f1_score(y_test, svm_predictions)
rf_f1_score = f1_score(y_test, rf_predictions)
gbdt_f1_score = f1_score(y_test, gbdt_predictions)

NameError: name 'pipeline' is not defined